##### 导入库

In [232]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import tree
from collections import Counter
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SVMSMOTE,SMOTE
from joblib import dump,load
from sklearn.model_selection import GridSearchCV

##### 选择数据

In [233]:
n='N_all.csv'
p='P_all.csv'
# p='P_ice.csv'
# p='P_her.csv'
# p='P_mix.csv'
# p='P_ice1.csv'
way="all"


##### 读取数据

In [234]:
fea_dict={
    'all':[
  "minvalue",
  "retang",
  "fft_abs_22",
  "fft_real_2",
  "lumpiness",
  "fft_real_5",
  "fft_angle_3",
  "alt_stderr_min_50",
  "fft_imag_6",
  "erbc_num10_focus2",
  "fft_imag_3",
  "alt_rvalue_min_50"
],


 'ice':[
  "retang",
  "erbc_num10_focus4",
  "fft_abs_22",
  "fft_real_2",
  "minindex",
  "alt_rvalue_min_50",
  "F2",
  "fft_imag_6",
  "asoc",
],

 'her':[
"fft_real_2",
"fft_real_5",
"dmq",
"fft_angle_3",
"erbc_num10_focus2",
"fft_imag_3",
"alt_rvalue_min_50"
],

'mix':[
"minvalue",
"cv",
"lumpiness",
"alt_rvalue_min_50",
"fft_angle_3",
"alt_stderr_min_50",
"maxindex"
]
}

fea_list=fea_dict[way]

def read_data(tit):
    path = 'E:\\desktop\\data\\app\\fea\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data

n_fea=read_data(n)
p_fea=read_data(p)

n_fea["tag"]=[0 for i in range(len(n_fea))]
p_fea["tag"]=[1 for i in range(len(p_fea))]

print(len(n_fea),len(p_fea))


523 360


##### 过采样

In [235]:
#过采样
# over=SMOTE(random_state=0)
over=SVMSMOTE(random_state=0)
np_fea=shuffle(pd.concat([n_fea,p_fea],axis=0))
x_data_=np_fea[fea_list]
y_data_=np_fea["tag"]

x_data,y_data=over.fit_resample(x_data_,y_data_)

#切分
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3)
print(x_data.shape)

(1046, 12)


In [236]:
def jugde(y_pred,y_true):
    tn,fn,tp,fp=0,0,0,0
    for i in range(len(y_pred)):
        
        if y_pred[i]==y_true[i]:
            if y_true[i]:
                tp+=1
            else:
                tn+=1
        else:
            if y_true[i]:
                fp+=1
            else:
                fn+=1

    print("检出率： ",tp,"/",tp+fp,' ',round(tp*100/(tp+fp),2),"%")
    print("误判率： ",fn,"/",tn+fn,' ',round(fn*100/(tn+fn),2),"%")
    print("精确率： ",(tn+tp),"/",(tn+fp+tp+fp),' ',round((tn+tp)*100/(tn+fp+tp+fp),2),"%")


##### 搭建模型

In [237]:

#导入模型存储和加载方法


#n_estimators决策树数量 RandomForestClassifier
# rfc=RandomForestClassifier(n_estimators=400,max_depth=20,class_weight='balanced') #海洛因
# rfc=RandomForestClassifier(n_estimators=700)
rfc=ExtraTreesClassifier()
# rfc=ExtraTreesClassifier(n_estimators=111,max_depth=8,max_features=11,random_state=0) #all
# rfc=RandomForestClassifier(n_estimators=381,max_depth=18,max_features=5,random_state=0) #mix
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)

print("测试集中：")
jugde(y_pred,list(y_test))

print("所有数据中：")
y_pred=rfc.predict(x_data_)
jugde(y_pred,list(y_data_))

rfc.score(x_test,y_test)

测试集中：
检出率：  134 / 157   85.35 %
误判率：  19 / 157   12.1 %
精确率：  272 / 318   85.53 %
所有数据中：
检出率：  343 / 360   95.28 %
误判率：  19 / 523   3.63 %
精确率：  847 / 881   96.14 %


0.8662420382165605

In [238]:
# #保存模型
# dump(rfc,'rfc_'+way+".joblib")
# print("保存成功")